In [ ]:
import random

def generate_matrix(rows, cols):
    matrix = []
    for _ in range(rows):
        row = [random.randint(1, 11) for _ in range(cols)]  # Generate random integers from 1 to 11 for the row
        matrix.append(row)
    return matrix

def write_matrices_to_file(matrix1, matrix2, filename):
    with open(filename, 'w') as file:
        # Write opcode and datatype
        file.write("1\n")
        file.write("1\n")

        # Write dimensions and contents of matrix 1
        matrix1_rows, matrix1_cols = len(matrix1), len(matrix1[0])
        file.write(f"{matrix1_rows}X{matrix1_cols}\n")
        for row in matrix1:
            row_str = " ".join(map(str, row))
            file.write(row_str + "\n")

        # Write dimensions and contents of matrix 2
        matrix2_rows, matrix2_cols = len(matrix2), len(matrix2[0])
        file.write(f"{matrix2_rows}X{matrix2_cols}\n")
        for row in matrix2:
            row_str = " ".join(map(str, row))
            file.write(row_str + "\n")

# Define matrix dimensions
matrix1_rows, matrix1_cols = 4096, 4096
matrix2_rows, matrix2_cols = 4096, 4096

# Generate matrices
matrix1 = generate_matrix(matrix1_rows, matrix1_cols)
matrix2 = generate_matrix(matrix2_rows, matrix2_cols)

# Write matrices to file
write_matrices_to_file(matrix1, matrix2, "matrices.txt")

print("Matrices generated and saved to file: matrices.txt")


Matrices generated and saved to file: matrices.txt


In [ ]:
import tensorflow as tf
import time

# Check if TPU is available
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("TPU not found")
    strategy = tf.distribute.get_strategy()

def read_matrix(file):
    # Read dimensions
    dimensions = file.readline().strip()
    rows, cols = map(int, dimensions.split('X'))

    # Read matrix
    matrix = []
    for _ in range(rows):
        row = list(map(float, file.readline().split()))
        matrix.append(row)

    return tf.constant(matrix)

# Read matrices from file
with open('matrices.txt', 'r') as file:
    # Skip opcode and datatype lines
    file.readline()
    file.readline()

    # Read first matrix
    A = read_matrix(file)

    # Read second matrix
    B = read_matrix(file)

# Use TensorFlow's TPU strategy
with strategy.scope():
    # Measure execution time
    start_time = time.time()

    # Perform matrix multiplication
    C = tf.linalg.matmul(A, B)
    # Calculate execution time
    end_time = time.time()
    execution_time = end_time - start_time

# Print the result
print("Result:")
with open('result.txt', 'w') as file:
    file.write("Result:\n")
    file.write(str(C.numpy()) + "\n\n")
print(C)
print("\nExecution time:", execution_time, "seconds")


TPU not found
Result:
tf.Tensor(
[[150535. 147428. 148241. ... 150434. 148728. 148058.]
 [151675. 149271. 147891. ... 150413. 150150. 147773.]
 [150152. 146277. 148867. ... 148965. 148206. 148158.]
 ...
 [149157. 145643. 147063. ... 148403. 147039. 146475.]
 [148058. 146959. 145969. ... 148299. 146450. 145978.]
 [149417. 147700. 149412. ... 150333. 149470. 149614.]], shape=(4096, 4096), dtype=float32)

Execution time: 0.0004963874816894531 seconds


In [ ]:
import tensorflow as tf
import random
import time
import numpy as np
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


def generate_matrix(rows, cols):
    matrix = []
    for _ in range(rows):
        row = [random.randint(1, 11) for _ in range(cols)]  # Generate random integers from 1 to 11 for the row
        matrix.append(row)
    return matrix

def write_matrices_to_file(matrix1, matrix2, filename):
    with open(filename, 'w') as file:
        # Write opcode and datatype
        file.write("1\n")
        file.write("1\n")

        # Write dimensions and contents of matrix 1
        matrix1_rows, matrix1_cols = len(matrix1), len(matrix1[0])
        file.write(f"{matrix1_rows}X{matrix1_cols}\n")
        for row in matrix1:
            row_str = " ".join(map(str, row))
            file.write(row_str + "\n")

        # Write dimensions and contents of matrix 2
        matrix2_rows, matrix2_cols = len(matrix2), len(matrix2[0])
        file.write(f"{matrix2_rows}X{matrix2_cols}\n")
        for row in matrix2:
            row_str = " ".join(map(str, row))
            file.write(row_str + "\n")

def read_matrix(file):
    # Read dimensions
    dimensions = file.readline().strip()
    rows, cols = map(int, dimensions.split('X'))

    # Read matrix
    matrix = []
    for _ in range(rows):
        row = list(map(int, file.readline().split()))
        matrix.append(row)

    return tf.constant(matrix)

# Generate matrices
#matrix1_rows, matrix1_cols = 4096, 4096
#matrix2_rows, matrix2_cols = 4096, 4096
#matrix1 = generate_matrix(matrix1_rows, matrix1_cols)
#matrix2 = generate_matrix(matrix2_rows, matrix2_cols)

# Write matrices to file
write_matrices_to_file(matrix1, matrix2, "matrices.txt")
print("Matrices generated and saved to file: matrices.txt")

# Read matrices from file
with open('matrices.txt', 'r') as file:
    # Skip opcode and datatype lines
    file.readline()
    file.readline()

    # Read first matrix
    A = read_matrix(file)

    # Read second matrix
    B = read_matrix(file)

tf.debugging.set_log_device_placement(True)

# Measure execution time
start_time = time.time()

# Perform matrix multiplication
C = tf.matmul(A, B)

# Calculate execution time
end_time = time.time()
execution_time = end_time - start_time

# Write result to file
np.savetxt('result.txt', C.numpy(), fmt='%d')

print("\nExecution time: " + str(execution_time) + " seconds")

print("Result written to file: result.txt")


Num GPUs Available:  0


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


Num GPUs Available:  1
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-imex' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2



--2024-04-24 14:11:54--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2024-04-24 14:11:54--  https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?OyS73Oc8-jj3wY0oFLCNfS8dpwyO-fg54Y60Brq0gkeuVt0FyPW1Ft8dQTXt56JcA5dDLPvJ2R-YXGjkjKOlUSXUJ3lw6ymfzLg2a

In [ ]:
!nvcc --version


/bin/bash: line 1: nvcc: command not found
